In [1]:
import regex as re
import polars as pl

import datetime
from datetime import date
import numpy as np
from scipy.stats import gaussian_kde

import numpy as np
import polars as pl
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import pickle as pkl

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt


from collections import Counter

# auto reload
%load_ext autoreload
%autoreload 2


In [2]:
from preprocess import preprocess
from analyze import analyze, plot_charts
info = preprocess("./chats/lucas.txt")
info = analyze(info)
# plot_charts(info)
chat_name = info["chat_name"]
df = info["df"]

In [ ]:
info

In [ ]:
"".join(df["message"].to_list())[:1050]

In [ ]:
info

In [4]:
author_unique = sorted(df["author"].unique().to_list())
# Define custom BuGy (blue → gray)
colorscale = [
    "#053061",  # dark navy
    "#2166ac",  # medium blue
    "#4393c3",  # light blue
    "#92c5de",  # very light blue
    # "#d1e5f0",  # pale blue
    # "#f7f7f7",  # very pale blue
    "#bababa",  # mediumgray
    "#878787",  # dark gray
    "#4d4d4d",  # very dark gray
    "#1c1c1c",  # near black
]
# colorscale = px.colors.sequential.Blues
colors = px.colors.sample_colorscale(colorscale, np.linspace(0, 1, len(author_unique)))
author_color_map = {author: color for author, color in zip(author_unique, colors)}

# graph title font
text_font = dict(family="Roboto", size=14, color="#333")
title_font = dict(family="Roboto Black", size=22, color="#333")

def pie_chart(values, labels, title, **kwargs):
    fig = px.pie(names=labels, values=values, title=title, width=512, height=512, hole=.3, **(kwargs))
    # display total amount in center
    total = sum(values)
    fig.add_annotation(dict(text=f'Total<br><b>{total}</b>', x=0.5, y=0.5, font_size=18, showarrow=False))
    fig.update_traces(textinfo='label+percent+value')#, insidetextorientation='horizontal', textposition='inside')
    fig.update_layout(showlegend=False, plot_bgcolor='white', 
        autosize=True,
        margin=dict(t=65, b=30, l=30, r=30),
        title_x=0.5,
        title_y=0.95,
        font=text_font,
        title_font=title_font
    )
    fig.show()
def bar_chart(x, y, title, x_title, y_title, **kwargs):
    fig = px.bar(x=x, y=y, title=title, width=512, height=512, **(kwargs))
    fig.update_layout(plot_bgcolor='white', xaxis_title=x_title, yaxis_title=y_title,
        showlegend=False,
        autosize=True,
        margin=dict(t=50, b=60, l=65, r=20),
        title_x=0.5,
        title_y=0.95,
        font=text_font,
        title_font=title_font
    )
    # display count on top of bars in x.x format
    fig.update_traces(texttemplate='%{y:.1f}', textposition='outside')
    fig.show()
    


In [ ]:
# import plotly.express as px
# import plotly.io as pio

# pio.kaleido.scope.default_format = "svg"
# pio.kaleido.scope.default_width = 700
# pio.kaleido.scope.default_height = 600

# data = {
#     "Author": ["Flo Klein", "Florian Hunecke", "Lucas Schnack"],
#     "Messages": [26822, 15918, 16378],
# }

# font_main = dict(family="Roboto, sans-serif", size=16, color="#222")
# font_title = dict(family="Roboto Black, sans-serif", size=22, color="#222")

# fig = px.pie(
#     data,
#     names="Author",
#     values="Messages",
#     hole=0.5,
#     title="Share of Messages per Author",
# )

# fig.update_traces(
#     texttemplate="%{label}<br>%{value:,}<br>%{percent:.1%}",
#     textposition="inside",
#     textfont=font_main,
# )

# fig.update_layout(
#     font=font_main,
#     title_font=font_title,
#     showlegend=False,
#     annotations=[dict(text="Total<br>59 118", font=font_title, showarrow=False)],
# )

# fig.write_image("piechart.svg")


In [ ]:
# calculate number of messages per author and anteil
anteile = df.group_by("author").len().with_columns(
    (pl.col("len") / pl.col("len").sum() * 100).alias("anteil_percent")
).sort("anteil_percent")

pie_chart(anteile["len"], 
          anteile["author"], 
          "Share of Messages per Author", 
          color_discrete_map=author_color_map, 
          color=anteile["author"])

In [ ]:
# average length of messages per author
avg_length = df.group_by("author").agg(
    pl.col("message_length").mean().round(1).alias("avg_message_length")
).sort("avg_message_length", descending=True)

bar_chart(avg_length["author"], 
    avg_length["avg_message_length"], 
    "Average Message Length per Author", 
    "Autor", 
    "Zeichen", 
    color=avg_length["author"], color_discrete_map=author_color_map,
    range_y=[avg_length["avg_message_length"].max()*0.8, avg_length["avg_message_length"].max() * 1.05]
)

In [ ]:
# total days chatted
chat_start = df["datetime"].min().date()
chat_end = df["datetime"].max().date()
total_days = chat_end - chat_start
total_days = total_days.days + 1  # include both start and end date
total_days

In [ ]:
df.with_columns(
    time_diff = pl.col("datetime").diff().over("author")
).group_by("author").agg([
    # total time in chat: sum of diffs < 1 minute
    pl.when(pl.col("time_diff") < datetime.timedelta(minutes=1))
        .then(pl.col("time_diff"))
        .sum()
        .alias("sum_time_diff"),

    # number of sessions: count of diffs > 15 minutes
    pl.when(pl.col("time_diff") > datetime.timedelta(minutes=15))
        .then(1)
        .sum()
        .alias("sessions"),
        
    # total active days per author
    (pl.col("datetime").dt.date().n_unique()).alias("active_days")
]).with_columns(
    (pl.col("sessions") / total_days).alias("avg_sessions_per_day")
)


In [ ]:
df.select(pl.col("datetime").diff())

In [ ]:
# messages per day 
df.group_by(pl.col("datetime").dt.date()).agg(
    pl.count().alias("message_count")
).mean()

In [ ]:
# get all word counts
all_text = " ".join(df["message"].to_list()).lower()
# split by word boundaries but keep if <word>
all_words = re.findall(r'\b\w+\b|<\w+>', all_text)

word_counts = Counter(all_words)
common_words = word_counts.most_common(20)
words, counts = zip(*common_words)
px.bar(x=words, y=counts, title="Häufigste Wörter im Chat", labels={"x": "Wörter", "y": "Anzahl"}, width=800, height=500).show()

In [ ]:
# filter for nomen using spacy
import spacy
nlp = spacy.load("de_core_news_sm")
all_unique_words = list(word_counts.keys())
all_unique_words_corpus = nlp(" ".join(all_unique_words))
nouns = [token.text for token in all_unique_words_corpus if token.pos_ == "NOUN" and token.is_alpha and len(token.text) > 2]
# nouns = []

In [ ]:
# plot nouns
special_include = ["dega"]
special_exclude = ["mal", "hab"]
nouns = list(set(nouns + special_include) - set(special_exclude))
noun_counts = {word: count for word, count in word_counts.items() if word in nouns}
common_nouns = Counter(noun_counts).most_common(30)
n, nc = zip(*common_nouns)
px.bar(x=n, y=nc, title="Häufigste Nomen im Chat", labels={"x": "Nomen", "y": "Anzahl"}, width=800, height=500).show()

In [ ]:
# word counts per author in subplots next to each other
import plotly.subplots as sp
authors = df.select(pl.col("author").unique()).to_series().to_list()
fig = sp.make_subplots(rows=1, cols=len(authors), subplot_titles=authors)
for i, author in enumerate(authors):
    author_messages = df.filter(pl.col("author") == author)
    author_words = " ".join(author_messages["message"].to_list()).lower().split()
    author_word_counts = Counter(author_words)
    common_author_words = author_word_counts.most_common(10)
    words, counts = zip(*common_author_words) if common_author_words else ([], [])
    fig.add_trace(go.Bar(x=words, y=counts, name=author, marker_color=author_color_map.get(author, "gray")), row=1, col=i+1)
fig.update_layout(title_text="Häufigste Wörter pro Autor", showlegend=False, width=400*len(authors), height=500,
                  plot_bgcolor='white')
fig.show()

In [ ]:
# noun count per author in subplots next to each other
fig = sp.make_subplots(rows=1, cols=len(authors), subplot_titles=authors)
for i, author in enumerate(authors):
    author_messages = df.filter(pl.col("author") == author)
    author_words = " ".join(author_messages["message"].to_list()).lower().split()
    author_word_counts = Counter(author_words)
    author_noun_counts = {word: count for word, count in author_word_counts.items() if word in nouns}
    common_author_nouns = Counter(author_noun_counts).most_common(10)
    n, nc = zip(*common_author_nouns) if common_author_nouns else ([], [])
    fig.add_trace(go.Bar(x=n, y=nc, name=author, marker_color=author_color_map.get(author, "gray")), row=1, col=i+1)
fig.update_layout(title_text="Häufigste \"Nomen\" pro Autor", showlegend=False, width=400*len(authors), height=500, 
                  plot_bgcolor='white')
fig.show()

In [ ]:
import pandas as pd 
# count days without messages 
total_days = pd.date_range(start=df.select(pl.col("datetime").min()).to_series()[0].date(), 
                           end=df.select(pl.col("datetime").max()).to_series()[0].date(), freq='D')
message_days = pd.to_datetime(df.select(pl.col("datetime").dt.date()).to_series().unique())
days_without_messages = total_days.difference(message_days)
print(f"Days without messages are:")
for d in days_without_messages:
    print(d.strftime("%Y-%m-%d"))

In [ ]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

In [ ]:
# do a continuous distribution of messages per time of day
hourly_df = df.with_columns(
    # hour of day column
    (pl.col("datetime").dt.hour() + pl.col("datetime").dt.minute() / 60.0).alias("hour_of_day")
)
# calculate kde of hour_of_day with scipy and cyclic wrap at 24 hours

# get authors
authors = hourly_df.select(pl.col("author").unique()).to_series().to_list()

# main plotly figure
fig = go.Figure()

# KDEs
x = np.linspace(0, 24, 200)
for author in authors:
    author_hours = hourly_df.filter(pl.col("author") == author)["hour_of_day"].to_numpy()
    if len(author_hours) < 2:
        continue
    kde = gaussian_kde(author_hours, bw_method="scott")
    y = kde(x)
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            name=author,
            line=dict(width=2, color=author_color_map[author]),
            #color=author_color_map[author],
            fill="tozeroy",
            # make fill more transparent
            fillcolor=author_color_map[author].__str__().replace("rgb", "rgba").replace(")", ",0.2)"),
        )
    )

# layout styling
fig.update_layout(
    title="Distribution of Messages over the Day",
    xaxis_title="Hour of the Day",
    yaxis_title="Density",
    template="plotly_white",
    width=1500,
    height=350,
    plot_bgcolor='white',
    xaxis=dict(visible=True, 
               showgrid=False, 
               zeroline=False,
               tick0=0, 
               dtick=2),
    yaxis=dict(visible=False),
    margin=dict(t=70, b=60, l=25, r=25),
    font=text_font,
    title_font=title_font,
    title_x=0.5,
    title_y=0.9,
)
# do legend within the plot
fig.update_layout(legend=dict(
    bgcolor="rgba(255, 255, 255, 0.8)",
    # position top left
    x=0,
    y=1
))

fig.show()


In [ ]:
# do a heatmap over all days like the github map
# y = day of week, x = week number of year, color = number of messages
from datetime import date
import datetime

# only take last 365 days
cutoff_date = df.select(pl.col("datetime").max()).to_series()[0] - datetime.timedelta(days=365)
# filter(pl.col("datetime") >= cutoff_date)
messages_per_day = df.with_columns(
    pl.col("datetime").dt.date().alias("date"),
).group_by("date").agg(
    pl.len().alias("message_count")
).with_columns(
    pl.col("date").dt.weekday().alias("day_of_week"),
    pl.col("date").dt.strftime("%G-%V").alias("year_week")
)

# Convert to Monday of that ISO week for continuous x-axis
messages_per_day = messages_per_day.with_columns(
    pl.col("year_week").map_elements(
        lambda s: date.fromisocalendar(int(s[:4]), int(s[5:]), 1),  # Monday of ISO week
        return_dtype=pl.Date
    ).alias("week_start")
)

# do a discrete color scale of continuous blues with 1000 steps
colorscale = px.colors.sample_colorscale(px.colors.sequential.Blues, np.linspace(0, 1, 1000))
heatmap_data = messages_per_day.select(["day_of_week", "week_start", "message_count"]).to_pandas()
heatmap_pivot = heatmap_data.pivot(index="day_of_week", columns="week_start", values="message_count").fillna(0)
# make np 2d array
heatmap_array = np.array(heatmap_pivot)
# map each entry to 3 channels according to colorscale
heatmap_colors = np.zeros((*heatmap_array.shape, 3), dtype=int)
for i in range(heatmap_array.shape[0]):
    for j in range(heatmap_array.shape[1]):
        heatmap_colors[i, j] = colorscale[((heatmap_array[i, j] - heatmap_pivot.min().min()) / (heatmap_pivot.max().max() - heatmap_pivot.min().min()) * 999).astype(int)].replace("rgb(","").replace(")", "").split(", ")


# map each value in heatmap_pivot to a discrete color in given colorscale, norm min and max to 0 and 1000
# heatmap_colors = heatmap_pivot.map(lambda x: colorscale[((x - heatmap_pivot.min().min()) / (heatmap_pivot.max().max() - heatmap_pivot.min().min()) * 999).astype(int)].replace("rgb(","").replace(")", "").split(", "))
# heatmap_colors = heatmap_colors.map(lambda x: np.array(x))
# heatmap_colors = np.array(heatmap_colors).reshape(heatmap_pivot.shape[0], heatmap_pivot.shape[1], 3)
# print(heatmap_colors)
# Continuous x-axis
fig = px.imshow(
    heatmap_colors,
    labels=dict(y="Wochentag", color="Anzahl Nachrichten"),
    x=heatmap_pivot.columns,
    y=["Mon  ", "Tue  ", "Wed  ", "Thu  ", "Fri  ", "Sat  ", "Sun  "],
    title="Messages per Day",
    width=1500,
    height=350,
    # color_continuous_scale="blues",
    # aspect="equal",
)
fig.update_layout(
    plot_bgcolor='white',
    autosize=True,
    margin=dict(t=60, b=50, l=0, r=120),
    title_x=0.5,
    title_y=0.95,
    font=text_font,
    title_font=title_font,
    # no x label
    xaxis_title="",
    yaxis_title="",
)
# no legend title
fig.update_coloraxes(colorbar_title="")

# Show only one tick per month
fig.update_xaxes(
    dtick="M1",        # 1 month interval
    tickformat="%b\n%Y",  # e.g. "Jan\n2024"
    
)
import os
fig.write_image(os.path.join("plots", "test.pdf"), format="pdf", scale=10)
fig.show()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load model and tokenizer once
MODEL_NAME = "oliverguhr/german-sentiment-bert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

@torch.inference_mode()
def sentiment_scores(texts: list[str], batch_size: int = 32):
    """
    Compute smooth sentiment scores for a list of German texts.
    Returns one float per text in [-1, 1].
    """
    all_scores = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch,
            truncation=True,
            padding=True,
            return_tensors="pt"
        ).to(device)

        logits = model(**enc).logits
        probs = F.softmax(logits, dim=-1)

        # Map from id to label
        id2label = model.config.id2label
        p_pos = probs[:, list(id2label.values()).index("positive")]
        p_neg = probs[:, list(id2label.values()).index("negative")]
        p_neu = probs[:, list(id2label.values()).index("neutral")]

        # Continuous sentiment score
        # scores = p_pos - p_neg 
        # H_p = - (p_pos * torch.log2(p_pos + 1e-12) + p_neg * torch.log2(p_neg + 1e-12) + p_neu * torch.log2(p_neu + 1e-12))
        # H_max = torch.log2(torch.tensor(3.0))
        # scores = (p_pos-p_neg) * (1- H_p/H_max)
        scores = torch.tanh(1.5 * ((p_pos - p_neg) * (1 - p_neu)))
        # scores = p_pos
        all_scores.extend(scores.cpu().tolist())

    return all_scores


In [ ]:
test = df.group_by_dynamic(
    index_column="datetime",
    every="12h",
).agg(
    pl.col("message").str.join("\n").alias("messages_per_window")
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# --- Configurable parameters ---
GERMAN_MODEL = "oliverguhr/german-sentiment-bert"
TEMPERATURE = 1  # flatten softmax; higher = smoother
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Load models/tokenizers ---
tok_de = AutoTokenizer.from_pretrained(GERMAN_MODEL)
model_de = AutoModelForSequenceClassification.from_pretrained(GERMAN_MODEL).to(DEVICE).eval()


@torch.inference_mode()
def sentiment_scores(texts: list[str], batch_size: int = 32, multilingual_fallback: bool = True):
    """
    Compute smooth sentiment scores in [-1, 1] for a list of texts.
    Combines temperature scaling, entropy weighting, and multilingual fallback.
    """
    all_scores = []

    def process_batch(model, tokenizer, batch):
        enc = tokenizer(
            batch,
            truncation=True,
            padding=True,
            return_tensors="pt"
        ).to(DEVICE)

        # Temperature-scaled logits (flatten sharp distributions)
        logits = model(**enc).logits / TEMPERATURE
        probs = F.softmax(logits, dim=-1)

        id2label = model.config.id2label
        try:
            p_pos = probs[:, list(id2label.values()).index("positive")]
            p_neg = probs[:, list(id2label.values()).index("negative")]
            p_neu = probs[:, list(id2label.values()).index("neutral")]
        except ValueError:
            # multilingual fallback uses labels POSITIVE / NEGATIVE / NEUTRAL uppercase
            p_pos = probs[:, list(id2label.values()).index("POSITIVE")]
            p_neg = probs[:, list(id2label.values()).index("NEGATIVE")]
            p_neu = probs[:, list(id2label.values()).index("NEUTRAL")]

        # Entropy weighting (confidence-based smoothing)
        entropy = -(probs * (probs + 1e-12).log()).sum(dim=-1)
        entropy_norm = entropy / torch.log(torch.tensor(probs.shape[1], device=DEVICE))
        weight = 1 - entropy_norm  # downweight uncertain predictions

        # Continuous score: tanh-scaled difference, entropy-weighted
        raw_score = p_pos - p_neg
        score = torch.tanh(1.5 * raw_score) * weight
        return score.cpu().tolist()

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        scores = process_batch(model_de, tok_de, batch)
        all_scores.extend(scores)

    return all_scores

sentiments = sentiment_scores(test["messages_per_window"].to_list(), batch_size=64)
# test = test.with_columns(pl.Series(sentiments).alias("sentiment_score"))

In [ ]:
# from germansentiment import SentimentModel
# model = SentimentModel()
# model.predict_sentiment(["Das ist ja großartig!", "Das ist ja schrecklich!"*2000], output_probabilities=True)[1]

# def get_sentiment(msgs):
#     results = model.predict_sentiment(msgs, output_probabilities=True)[1]
#     calc_scores = lambda res : (res[0][1] - res[1][1])
#     scores = [calc_scores(res) for res in results]
#     return scores



# sentiments_per_week = sentiment_scores(test["messages_per_window"].to_list(), batch_size=64)

test = test.with_columns(
    pl.Series(sentiments).alias("sentiment_score"),
).with_columns(
    pl.col("sentiment_score").rolling_mean(window_size=28, min_samples=1).alias("sentiment_rolling")
)   

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=test["datetime"],
        y=test["sentiment_rolling"],
        mode="lines",
        name="14-Tage gleitender Durchschnitt",
        line=dict(color="MediumPurple", width=4)
    )
)   
fig.add_trace(
    go.Scatter(
        x=test["datetime"],
        y=test["sentiment_score"],
        mode="markers",
        name="Täglicher Sentiment Score",
        marker=dict(color="LightSkyBlue", size=6, opacity=0.6)
    )
)
fig.update_layout(
    title="Sentiment Verlauf über die Zeit",
    xaxis_title="Datum",
    yaxis_title="Sentiment Score",
    template="plotly_white",
    width=900,
    height=500,
)
fig.show()

In [ ]:
word_counts["<bild>"]

In [ ]:
all_mediums = {m: "<" + m.lower() + ">" for m in mediums} | {"Sprachanruf": "sprachanruf"}

medium_counter = {medium: word_counts[all_mediums[medium]] for medium in all_mediums}

medium_counter_df = pl.DataFrame(list(medium_counter.items()), schema=["medium", "count"], orient="row").sort("count", descending=True)
# plot bar chart of medium_counter
px.bar(x=medium_counter_df["medium"], y=medium_counter_df["count"], 
       title="Anzahl der Medien im Chat",
       labels={"x": "Medientyp", "y": "Anzahl"}, width=800, height=500, log_y=True).show()

In [ ]:
word_counts["ich"]

In [ ]:
# sentiment over time using textblob
# from transformers import pipeline
# sentiment_analyzer = pipeline("sentiment-analysis", model="oliverguhr/german-sentiment-bert")
# def get_sentiment(messages):
#     msgs = [msg[:512] for msg in messages]  # truncate to 512 chars
#     results = sentiment_analyzer(msgs)
#     scores = [result['score'] * (1 if result['label'] == 'POSITIVE' else -1) for result in results]
#     return scores



In [ ]:
# def get_last_monday(date):
#     return (date - datetime.timedelta(days=date.weekday())).date()

# sentiment_per_week = df.with_columns(
#     pl.col('datetime').map_elements(get_last_monday, return_dtype=pl.Date).alias('last_monday'),
# ).group_by("last_monday").agg(
#     pl.concat_str("message").alias("messages_per_week")
# ).sort("last_monday")
# sentiment_per_week

In [ ]:
# average time between messages
df.select(pl.col("datetime").diff().mean()), df.select(pl.col("datetime").diff().median())

In [ ]:
df.select(pl.col("datetime").diff()).filter(pl.col("datetime").diff() < datetime.timedelta(hours=2)).mean()

# IDeen

antwort delays messen

llm je author trainieren

emojis

konversationen clustern
clustern zu topics ordnen



In [ ]:
import polars as pl
import plotly.express as px
import datetime
from datetime import date

def heatmap_chart(
    df: pl.DataFrame,
    time_col: str,
    count_col: str | None = None,
    title: str = "Heatmap",
    xaxis_name: str = "",
    yaxis_name: str = "",
    save_path: str | None = None,
    days_back: int = 365,
    color_scale: str = "Blues",
    width: int = 1500,
    height: int = 512,
    text_font: dict | None = None,
    title_font: dict | None = None,
):
    """
    Creates a GitHub-style calendar heatmap over the last N days.

    Parameters
    ----------
    df : pl.DataFrame
        Input dataframe.
    time_col : str
        Name of the datetime column.
    count_col : str | None
        Optional name for the column to count occurrences of. If None, counts rows.
    title : str
        Plot title.
    xaxis_name, yaxis_name : str
        Axis labels.
    save_path : str | None
        Path to save the figure (optional).
    days_back : int
        Number of days to include (default 365).
    color_scale : str
        Continuous color map name.
    width, height : int
        Figure dimensions.
    text_font, title_font : dict | None
        Font dictionaries for customization.
    """

    cutoff_date = df.select(pl.col(time_col).max()).to_series()[0] - datetime.timedelta(days=days_back)
    df_filtered = df.filter(pl.col(time_col) >= cutoff_date)

    # Aggregate message counts per day
    messages_per_day = (
        df_filtered
        .with_columns(pl.col(time_col).dt.date().alias("date"))
        .group_by("date")
        .agg(pl.len().alias("message_count") if count_col is None else pl.count(count_col).alias("message_count"))
        .with_columns(
            pl.col("date").dt.weekday().alias("day_of_week"),
            pl.col("date").dt.strftime("%G-%V").alias("year_week"),
        )
    )

    # Convert week identifier to Monday of ISO week for continuous axis
    messages_per_day = messages_per_day.with_columns(
        pl.col("year_week").map_elements(
            lambda s: date.fromisocalendar(int(s[:4]), int(s[5:]), 1),
            return_dtype=pl.Date,
        ).alias("week_start")
    )

    # Pivot for heatmap
    heatmap_data = messages_per_day.select(["day_of_week", "week_start", "message_count"]).to_pandas()
    heatmap_pivot = heatmap_data.pivot(index="day_of_week", columns="week_start", values="message_count").fillna(0)

    # Plot
    fig = px.imshow(
        heatmap_pivot,
        labels=dict(x=xaxis_name, y=yaxis_name, color="Anzahl"),
        x=heatmap_pivot.columns,
        y=["Mon  ", "Tue  ", "Wed  ", "Thu  ", "Fri  ", "Sat  ", "Sun  "],
        title=title,
        width=width,
        height=height,
        color_continuous_scale=color_scale,
    )
    fig.update_layout(
        plot_bgcolor='white',
        autosize=True,
        margin=dict(t=60, b=50, l=0, r=120),
        title_x=0.5,
        title_y=0.95,
        font=text_font or dict(family="Roboto", size=14, color="#333"),
        title_font=title_font or dict(family="Roboto Black", size=22, color="#333"),
        xaxis_title="",
        yaxis_title="",
    )
    fig.update_coloraxes(colorbar_title="")
    fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")

    if save_path:
        fig.write_image(save_path)
    else:
        fig.show()

    return fig


In [ ]:
heatmap_chart(
    df,
    time_col="datetime",
    title="Messages per Day",
    save_path="messages_heatmap.svg",
    width=1500,
    height=350,
)


In [6]:
whole_text = " ".join(df["message"].to_list())
emojis = re.findall(r'\p{Extended_Pictographic}', whole_text)
# count emojis
emojis_counter = Counter(emojis)
common_emojis = emojis_counter.most_common(15)
e, ec = zip(*common_emojis)
bar_chart(e, ec, "Häufigste Emojis im Chat", "Emoji", "Anzahl")

In [31]:
x["message"].to_list()[2]

'Hoffe du hast doch jetzt nicht unbeliebt gemacht 😂'

In [36]:
# count emojis
import emoji
all_emojis = [c for c in "".join(df["message"].to_list()) if c in emoji.EMOJI_DATA]
emoji_counts = Counter(all_emojis)
common_emojis = emoji_counts.most_common(20)
emojis, counts = zip(*common_emojis)
px.bar(x=emojis, y=counts, title="Häufigste Emojis im Chat", labels={"x": "Emojis", "y": "Anzahl"}, width=800, height=500).show()

ModuleNotFoundError: No module named 'emoji'